In [1008]:
import cv2
import os
import pandas as pd
import random
import numpy as np
import time
import colorsys
from PIL import Image, ImageDraw, ImageFont
from cv2 import dnn_superres

In [1009]:
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename), cv2.IMREAD_UNCHANGED)
        if img is not None:
            images.append(img)
    return images

In [1010]:
def modify_small_face(face, brightness, filter_size):
    hsv = cv2.cvtColor(face, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(hsv)
    v = cv2.add(v,brightness)
    v[v > 255] = 255
    v[v < 0] = 0
    final_hsv = cv2.merge((h, s, v))
    brighter_face = cv2.cvtColor(final_hsv, cv2.COLOR_HSV2BGR)
    smoother_face = cv2.medianBlur(brighter_face, filter_size)
    #smoother_face = cv2.blur(brighter_face, (filter_size, filter_size))
    #smoother_face = cv2.GaussianBlur(brighter_face, (filter_size, filter_size), 0)
    return smoother_face

    

In [1011]:
def add_face_to_template(face, template, x_offset, y_offset, small=False):
    y1, y2 = y_offset, y_offset+face.shape[0]
    x1, x2 = x_offset, x_offset+face.shape[1]
    a = face[:,:,3]
    if small == True:
        face = modify_small_face(face[:,:,:3].copy(), 70, 3)
    face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
    gray_face_4ch = cv2.merge([face, face, face, a])
    alpha_s = gray_face_4ch[:, :, 3] / 255.0
    alpha_l = 1.0 - alpha_s

    for c in range(0, 3):
        template[y1:y2, x1:x2, c] = (alpha_s * gray_face_4ch[:, :, c] +
                                alpha_l * template[y1:y2, x1:x2, c])
    return template

In [1012]:
def crop_face(face, size, ratio_w, ratio_h):
    face = cv2.resize(face, (size, size), interpolation=cv2.INTER_AREA)
    ratio = ratio_w/ratio_h
    nw = ratio*face.shape[1] #height, width
    nw2 = (face.shape[1] - nw)/2
    face = face[:, int(nw2):face.shape[0]-int(nw2)]
    return face

In [1013]:
def add_text_PIL(Id, text, x_offset, y_offset, size, color='black', font='font/arialbd.ttf'):
    pil_image = Image.fromarray(cv2.cvtColor(Id, cv2.COLOR_BGR2RGB))

    draw = ImageDraw.Draw(pil_image)

    font = ImageFont.truetype(font, size)  
    draw.text((x_offset, y_offset), text, font=font, fill=color)

    Id = cv2.cvtColor(np.array(pil_image), cv2.COLOR_RGB2BGR)
    return Id

def add_name_PIL(Id, text, y_offset, size, color='black', font='font/arialbd.ttf'):
    pil_image = Image.fromarray(cv2.cvtColor(Id, cv2.COLOR_BGR2RGB))
    font = ImageFont.truetype(font, size)  
    draw = ImageDraw.Draw(pil_image)
    ###
    _, _, w, h = draw.textbbox((0, 0), text, font=font)
    draw.text(((Id.shape[1]-w)/2, (Id.shape[0]-h)/2 + y_offset), text, font=font, fill=color)
    ###
    
    # draw.text((x_offset, y_offset), text, font=font, fill=color)

    Id = cv2.cvtColor(np.array(pil_image), cv2.COLOR_RGB2BGR)
    return Id

In [1014]:
def add_name_to_Id(Id, names, seed):
    random.seed(seed)
    i = random.randint(0, 199)
    name = names['IMIĘ 1'][i]

    # Id = add_text_PIL(Id, name, x_offset, y_offset, 20, 'black', 'font/arial.ttf')
    Id = add_name_PIL(Id, name, -40, 20, 'black', 'font/arial.ttf')
    return Id

def add_surname_to_Id(Id, surnames, seed):
    random.seed(seed)
    i = random.randint(0, 199)
    surname = surnames['nazwisko'][i]

    # Id = add_text_PIL(Id, surname, x_offset, y_offset, 20, 'black', 'font/arial.ttf')
    Id = add_name_PIL(Id, surname, -10, 20, 'black', 'font/arial.ttf')

    return Id


In [1015]:
def generate_birthdate(seed):
    start = "1 1 1971"
    end = "1 1 2006"
    time_format = "%d %m %Y"
    random.seed(seed)
    i = random.random()
    stime = time.mktime(time.strptime(start, time_format))
    etime = time.mktime(time.strptime(end, time_format))

    ptime = stime + i * (etime - stime)
    bday = time.strftime(time_format, time.localtime(ptime))

    return bday

def add_issue_date(Id, x_offset, y_offset, seed):
    start = "1 1 2020"
    end = "31 12 2023"
    time_format = "%d %m %Y"
    time_format2 = "%d.%m.%Y"
    random.seed(seed)
    i = random.random()
    stime = time.mktime(time.strptime(start, time_format))
    etime = time.mktime(time.strptime(end, time_format))

    ptime = stime + i * (etime - stime)

    issuedate = time.strftime(time_format2, time.localtime(ptime)) 
    issueyear = time.strftime('%Y', time.localtime(ptime)) 
    Id = add_text_PIL(Id, issuedate, x_offset, y_offset, 20, 'black', 'font/arial.ttf')

    return Id, issueyear



In [1016]:
def add_pesel(Id, x_offset, y_offset, seed):
    bday = generate_birthdate(seed)
    bday = bday.split(' ')
    year = int(bday[2][2:])
    month = int(bday[1])
    day = bday[0]
    if year <= 24:
        month += 20
    if month < 10:
        month = '0' + str(month)
    else:
        month = str(month)
    if year < 10:
        year = '0' + str(year)
    else:
        year = str(year)
    pesel = str(year) + month + day
    random.seed(seed)
    for _ in range(4):
        pesel += str(random.randint(0,9))

    pesel += random.choice('13579')
    Id = add_text_PIL(Id, pesel, x_offset, y_offset, 20, 'black', 'font/arial.ttf')
    

    return Id

def add_album(Id, issueyear, x_offset, y_offset, seed):
    random.seed(seed)
    album = str(int(issueyear[-1]) + 1)
    for _ in range(5):
        album += str(random.randint(0,9))
    Id = add_text_PIL(Id, album, x_offset, y_offset, 20, 'black', 'font/arial.ttf')

    return Id

In [1017]:
def add_agh(Id, x_off1, y_off1, x_off2, y_off2, x_off3, y_off3):
    Id = add_text_PIL(Id, "Akademia Górniczo-Hutnicza", x_off1, y_off1, 20, 'black', 'font/arial.ttf')
    Id = add_text_PIL(Id, "im. St. Staszica", x_off2, y_off2, 20, 'black', 'font/arial.ttf')
    Id = add_text_PIL(Id, "w Krakowie", x_off3, y_off3, 20, 'black', 'font/arial.ttf')
    return Id

In [1018]:
def upscale_template(template):
    h = template.shape[0] * 2
    w = template.shape[1] * 2

    template = cv2.resize(template, (w, h), interpolation=cv2.INTER_CUBIC)
    
    return template

In [1019]:
def add_mark(Id, x_offset, y_offset):
    text = 'FAŁSZYWA ELS'
    Id = add_text_PIL(Id, text, x_offset, y_offset, 30, 'red', 'font/arial.ttf')
    return Id

In [1020]:
def distortions(Id, seed):
    # rotation
    random.seed(seed)
    w, h = Id.shape[1], Id.shape[0]
    angle = random.uniform(-0.8,0.8)
    w_change = random.randint(-30, 30)
    h_change = random.randint(-30, 30)
    center = ((w + w_change) // 2, (h + h_change) // 2)
    rotation_matrix = cv2.getRotationMatrix2D(center, angle, 1.0)
    Id = cv2.warpAffine(Id, rotation_matrix, (w, h))

    # blur
    # if random.randint(1,5) % 5 == 1:
    #     Id = cv2.blur(Id, (3, 3))
    
    # brightness
    hsv = cv2.cvtColor(Id, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(hsv)
    brightness = random.randint(-50, 50)
    v = cv2.add(v,brightness)
    v[v > 255] = 255
    v[v < 0] = 0
    final_hsv = cv2.merge((h, s, v))
    Id = cv2.cvtColor(final_hsv, cv2.COLOR_HSV2BGR)
    
    # yellow filter
    if random.randint(0,1) % 2 == 1:
        h, s, v = 49, random.randint(60, 100), random.randint(40, 60)
        rgb = colorsys.hsv_to_rgb(h,s,v)
        yellow_filter = np.full_like(Id, rgb, dtype=np.uint8)
        alpha = 0.7
        beta = 0.1  
        gamma = 0 

        Id = cv2.addWeighted(Id, alpha, yellow_filter, beta, gamma)

    return Id

In [1021]:
def generate_identity(face_org, template, names, surnames, cities, seed):
    template = upscale_template(template)
    face = crop_face(face_org.copy(), 250, 35, 45)
    Id = add_face_to_template(face, template, 565, 220)
    Id = add_name_to_Id(Id, names, seed)
    Id = add_surname_to_Id(Id, surnames, seed)
    Id, issueyear = add_issue_date(Id, 160, 348, seed)
    Id = add_album(Id, issueyear, 160, 374,  seed)
    Id = add_pesel(Id, 160, 401, seed)
    Id = add_agh(Id, 290, 48, 409, 68, 447, 88)
    Id = add_mark(Id, 307, 400)
    Id = distortions(Id, seed)
    
    

    cv2.imwrite(f'IDs/ID_{seed}.jpg', Id)

In [1022]:
template = cv2.imread("template/template.jpg")
faces = load_images_from_folder("meskie_lica_bez_tla")
names = pd.read_csv("meskie_imiona/imiona.csv", nrows=200)
surnames = pd.read_csv("meskie_nazwiska/nazwiska.csv", nrows=200)
cities = pd.read_csv("miasta/wojewodztwa_miasta.csv", sep=';')

In [1023]:
for seed, face in enumerate(faces):
    generate_identity(face, template.copy(), names, surnames, cities, seed)